In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
from collections import Counter

데이터 가져오기

In [159]:
vod = pd.read_csv('vod_prepro_series_nm.csv')
con = pd.read_csv('con_prepro_series_nm.csv', encoding='cp949')
c_score=pd.read_csv('c_score.csv', encoding='cp949')

In [8]:
vod

,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,SMRY,ACTR_DISP,disp_rtm,strt_dt,Category,rename,vod_id
0,65941000,그것이알고싶다,TV 시사/교양,기타,80.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230812163507,TV프로그램,그것이알고싶다,297
1,66873000,그것이알고싶다,TV 시사/교양,기타,80.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230816205227,TV프로그램,그것이알고싶다,297
2,66873000,그것이알고싶다,TV 시사/교양,기타,12.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230829194727,TV프로그램,그것이알고싶다,297
3,61689000,그것이알고싶다,TV 시사/교양,기타,80.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230813130609,TV프로그램,그것이알고싶다,297
4,61619000,꼬리에꼬리를무는그날이야기,TV 시사/교양,기타,69.7,"살인범의 미토콘드리아 - 2006 냉동고 살인사건. 2006년 7월 23일, 서울 ...","장도연,장현성,장성규",73,20230804092737,TV프로그램,꼬리에꼬리를무는그날이야기,8
...,...,...,...,...,...,...,...,...,...,...,...,...
10156,60169000,간질간질,키즈,학습,6.0,인기작가 서현의 그림책! 온몸으로 읽는 그림책 간질간질. 글자가 형태로 보이지 않고...,-,6,20230914211110,키즈,간질간질,3674
10157,66317000,알사탕,키즈,학습,8.0,"인기 작가 백희나 신간. 교보문고, YES24 베스트셀러! 상대방의 입장을 헤아리고...",-,8,20230903223544,키즈,알사탕,3671
10158,66317000,알사탕,키즈,학습,3.3,"인기 작가 백희나 신간. 교보문고, YES24 베스트셀러! 상대방의 입장을 헤아리고...",-,8,20230903223210,키즈,알사탕,3671
10159,66900000,월간 아기상어,키즈,학습,0.8,8-Bit Baby Shark. 오락실에 아기 상어가 나타났다?! 추억의 8비트 버...,아기상어,1,20230904120057,키즈,월간아기상어,406


In [160]:
c_score

,subsr,series_nm,vod_id,Affinity,ct_cl,genre_of_ct_cl
0,59879000,나쁜엄마,3403,1,TV드라마,기타
1,59879000,나의 해방일지,1859,1,TV드라마,기타
2,59879000,낭만닥터 김사부2,1168,1,TV드라마,기타
3,59879000,소방서 옆 경찰서,2092,2,TV드라마,기타
4,59879000,슬기로운 의사생활 시즌2,2891,1,TV드라마,기타
...,...,...,...,...,...,...
7708,67160000,심야괴담회,143,1,TV 연예/오락,기타
7709,67160000,콘크리트 유토피아,2630,1,영화,드라마
7710,67161000,스파이 코드명 포춘,3824,2,영화,액션/어드벤쳐
7711,67164000,미션 임파서블: 데드 레코닝 PART ONE,2867,2,영화,액션/어드벤쳐


In [13]:
# 러닝타임 0인 행 확인 후 제거
vod[vod['disp_rtm'] == 0].shape[0]
vod_filtered = vod[vod['disp_rtm'] != 0].reset_index(drop=True)

In [14]:
# subsr 이상치 확인
vod_filtered['subsr'].value_counts()

subsr
64154000    795
66875000    545
60326000    530
64385000    520
66705000    363
           ... 
63038000      1
63387000      1
65104000      1
66402000      1
60040000      1
Name: count, Length: 406, dtype: int64

In [15]:
vod_filtered

,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,SMRY,ACTR_DISP,disp_rtm,strt_dt,Category,rename,vod_id
0,65941000,그것이알고싶다,TV 시사/교양,기타,80.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230812163507,TV프로그램,그것이알고싶다,297
1,66873000,그것이알고싶다,TV 시사/교양,기타,80.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230816205227,TV프로그램,그것이알고싶다,297
2,66873000,그것이알고싶다,TV 시사/교양,기타,12.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230829194727,TV프로그램,그것이알고싶다,297
3,61689000,그것이알고싶다,TV 시사/교양,기타,80.0,살인자의 자백 그리고 아크말의 고백. 방대한 수사기록과 당시 아크말의 진술을 토대로...,김상중,80,20230813130609,TV프로그램,그것이알고싶다,297
4,61619000,꼬리에꼬리를무는그날이야기,TV 시사/교양,기타,69.7,"살인범의 미토콘드리아 - 2006 냉동고 살인사건. 2006년 7월 23일, 서울 ...","장도연,장현성,장성규",73,20230804092737,TV프로그램,꼬리에꼬리를무는그날이야기,8
...,...,...,...,...,...,...,...,...,...,...,...,...
10156,60169000,간질간질,키즈,학습,6.0,인기작가 서현의 그림책! 온몸으로 읽는 그림책 간질간질. 글자가 형태로 보이지 않고...,-,6,20230914211110,키즈,간질간질,3674
10157,66317000,알사탕,키즈,학습,8.0,"인기 작가 백희나 신간. 교보문고, YES24 베스트셀러! 상대방의 입장을 헤아리고...",-,8,20230903223544,키즈,알사탕,3671
10158,66317000,알사탕,키즈,학습,3.3,"인기 작가 백희나 신간. 교보문고, YES24 베스트셀러! 상대방의 입장을 헤아리고...",-,8,20230903223210,키즈,알사탕,3671
10159,66900000,월간 아기상어,키즈,학습,0.8,8-Bit Baby Shark. 오락실에 아기 상어가 나타났다?! 추억의 8비트 버...,아기상어,1,20230904120057,키즈,월간아기상어,406


In [16]:
vod[(vod['subsr'] == 64154000) & vod['asset_nm'].str.contains('배달꾼')]

,subsr,asset_nm,ct_cl,genre_of_ct_cl,use_tms,SMRY,ACTR_DISP,disp_rtm,strt_dt,Category,rename,vod_id
1618,64154000,최강 배달꾼,TV드라마,기타,51.0,"단아는 진규의 도움으로 헬 조선 탈출의 희망을 품고, 강수는 배달부들과 함께 불법 ...","고경표,채수빈,김선호,고원희,조희봉,예수정,이민영,김기두,허지원,강봉성",51,20230817221110,TV프로그램,최강배달꾼,754
1619,64154000,최강 배달꾼,TV드라마,기타,51.0,"단아는 진규의 도움으로 헬 조선 탈출의 희망을 품고, 강수는 배달부들과 함께 불법 ...","고경표,채수빈,김선호,고원희,조희봉,예수정,이민영,김기두,허지원,강봉성",51,20230822034011,TV프로그램,최강배달꾼,754
1620,64154000,최강 배달꾼,TV드라마,기타,6.5,"단아는 진규의 도움으로 헬 조선 탈출의 희망을 품고, 강수는 배달부들과 함께 불법 ...","고경표,채수빈,김선호,고원희,조희봉,예수정,이민영,김기두,허지원,강봉성",51,20230831013822,TV프로그램,최강배달꾼,754
1621,64154000,최강 배달꾼,TV드라마,기타,40.7,"단아는 진규의 도움으로 헬 조선 탈출의 희망을 품고, 강수는 배달부들과 함께 불법 ...","고경표,채수빈,김선호,고원희,조희봉,예수정,이민영,김기두,허지원,강봉성",51,20230831023828,TV프로그램,최강배달꾼,754
1622,64154000,최강 배달꾼,TV드라마,기타,51.0,"단아는 진규의 도움으로 헬 조선 탈출의 희망을 품고, 강수는 배달부들과 함께 불법 ...","고경표,채수빈,김선호,고원희,조희봉,예수정,이민영,김기두,허지원,강봉성",51,20230808182409,TV프로그램,최강배달꾼,754
...,...,...,...,...,...,...,...,...,...,...,...,...
7809,64154000,최강 배달꾼,TV드라마,기타,7.8,"팔팔수타에 들이닥친 용철 일당은 장동수에게 거부할 수 없는 제안을 해오고, 그로 인...","고경표,채수빈,김선호,고원희,조희봉,예수정,이민영,김기두,허지원,강봉성",53,20230928052145,TV프로그램,최강배달꾼,754
7810,64154000,최강 배달꾼,TV드라마,기타,53.0,"팔팔수타에 들이닥친 용철 일당은 장동수에게 거부할 수 없는 제안을 해오고, 그로 인...","고경표,채수빈,김선호,고원희,조희봉,예수정,이민영,김기두,허지원,강봉성",53,20230919110835,TV프로그램,최강배달꾼,754
7811,64154000,최강 배달꾼,TV드라마,기타,53.0,"팔팔수타에 들이닥친 용철 일당은 장동수에게 거부할 수 없는 제안을 해오고, 그로 인...","고경표,채수빈,김선호,고원희,조희봉,예수정,이민영,김기두,허지원,강봉성",53,20230912102334,TV프로그램,최강배달꾼,754
7812,64154000,최강 배달꾼,TV드라마,기타,3.4,"팔팔수타에 들이닥친 용철 일당은 장동수에게 거부할 수 없는 제안을 해오고, 그로 인...","고경표,채수빈,김선호,고원희,조희봉,예수정,이민영,김기두,허지원,강봉성",53,20230919123614,TV프로그램,최강배달꾼,754


rating 정보 생성

In [19]:
# 사용자별 러닝타임 대비 시청한 시간 계산
vod_grouped = vod_filtered.groupby(['subsr', 'vod_id'])

In [23]:
# subsr, asset_nm, 평점 dataframe 생성
ratings = []
for group, data in vod_grouped:
    ratings.append([group[0], group[1], (data['use_tms']/data['disp_rtm']).mean()])
    print(group[1], (data['use_tms']/data['disp_rtm']).mean())

1190 0.15570314824845755
2092 0.20467943994104645
784 1.0
38 0.0019047619047619048
149 0.4908256880733945
297 1.0
572 1.0
613 0.0002949852507374632
1074 0.07701149425287357
2172 0.32704918032786884
2537 1.0
3314 0.4947368421052632
1267 0.08370786516853931
2172 0.09426229508196722
3576 0.102
4515 0.16166666666666665
456 0.885
728 0.3279488416988417
1190 0.20060240963855422
1270 0.993103448275862
2097 0.025180375180375178
2117 0.031578947368421054
2481 0.0773913043478261
2846 1.0
4442 1.0
296 0.26036084249895425
1267 0.010674157303370787
2431 0.002564102564102564
1329 0.0020408163265306124
2633 0.9559055118110237
360 0.9968641114982578
1673 0.5395833333333333
1953 0.11223021582733812
2677 0.0
3824 0.47982456140350876
4018 0.0
376 0.03414634146341463
381 0.5280966183574879
810 0.7675
849 0.4696876620527344
884 0.6863003663003663
887 0.6695767195767196
954 0.713458507601304
976 0.4464266190075694
992 0.3287062566277837
1136 0.45169098494214954
1149 0.32051282051282054
1346 0.50892621870882

930 1.0
1166 0.5
1215 0.2835793357933579
1641 0.0
2733 0.4575268817204301
2810 0.38818181818181824
2865 0.16705426356589148
3227 0.21194444444444446
4700 0.21595441595441592
2172 0.09016393442622951
3314 0.04868421052631579
3576 0.07200000000000001
4381 0.6127272727272728
747 0.543823738450604
476 0.7783532016691936
485 0.6934782608695652
538 0.2872695459201093
888 0.6888200128390306
1009 0.2456521739130435
1076 0.6573529411764706
1267 0.26292134831460673
1428 0.7706422018348624
1676 0.2
2172 0.07295081967213116
2349 0.02340425531914894
2533 0.08686868686868686
3314 0.5157894736842106
3530 0.0
4129 0.050980392156862744
1822 0.1
4021 0.0956140350877193
4442 0.7440265486725663
3917 0.0
7 0.0
7 0.05575221238938053
830 1.0
880 0.8022222222222222
1583 0.0038461538461538464
1591 1.0
296 0.939247311827957
29 0.8683431952662721
163 0.41751412429378537
252 0.9517857142857142
340 0.5373333333333333
850 1.0
1018 0.7155906593406594
1292 0.5425925925925926
1326 0.11945945945945946
1360 0.5033333333

4107 0.04903846153846154
7 0.48672566371681414
159 0.16499999999999998
2130 0.9027397260273974
2741 0.18805555555555556
2850 0.10077519379844961
3576 0.021
3951 0.26862745098039215
1215 0.24206642066420667
2537 0.1361904761904762
1676 0.02616822429906542
3314 0.005263157894736842
94 0.5083333333333333
1373 0.6
1399 0.5125
937 0.7545454545454545
1386 0.0328125
1398 0.0044444444444444444
2297 0.0
2850 0.12218246869409659
2988 0.003076923076923077
3623 1.0
3850 0.25209923664122136
3917 0.20408163265306123
313 0.03777777777777778
1550 0.5022727272727273
689 0.016666666666666666
296 1.0
30 0.6065671641791045
1191 0.5708336738330141
2288 0.13115763546798032
1267 0.1101123595505618
2297 0.6634615384615384
3623 0.3474358974358974
296 0.9090909090909091
449 0.03428571428571429
755 0.041304347826086954
1159 0.8294736842105263
1457 0.6277227722772277
2033 0.05309734513274336
2754 0.0
2846 0.14426229508196722
3339 0.35327102803738314
3850 0.9740458015267175
4689 0.0467741935483871
289 0.875
421 0.

In [24]:
ratings = pd.DataFrame(ratings, columns=['subsr', 'vod_id', 'rating'])
ratings

,subsr,vod_id,rating
0,59879000,1190,0.155703
1,59879000,2092,0.204679
2,59895000,784,1.000000
3,59900000,38,0.001905
4,59900000,149,0.490826
...,...,...,...
2166,67148000,689,0.091667
2167,67148000,880,0.931818
2168,67154000,476,0.009160
2169,67161000,3824,0.001754


In [25]:
# rating을 1~5까지 분류
bins = [0, 0.2, 0.4, 0.6, 0.8, 1.0]
labels = [1,2,3,4,5]
ratings['stars'] = pd.cut(ratings['rating'], bins=bins, labels=labels, include_lowest=True)
ratings['stars'] = ratings['stars'].astype(int)

In [26]:
ratings.info()
ratings

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2171 entries, 0 to 2170
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   subsr   2171 non-null   int64  
 1   vod_id  2171 non-null   int64  
 2   rating  2171 non-null   float64
 3   stars   2171 non-null   int32  
dtypes: float64(1), int32(1), int64(2)
memory usage: 59.5 KB


,subsr,vod_id,rating,stars
0,59879000,1190,0.155703,1
1,59879000,2092,0.204679,2
2,59895000,784,1.000000,5
3,59900000,38,0.001905,1
4,59900000,149,0.490826,3
...,...,...,...,...
2166,67148000,689,0.091667,1
2167,67148000,880,0.931818,5
2168,67154000,476,0.009160,1
2169,67161000,3824,0.001754,1


vod list 만들기

In [4]:
vod_list = pd.read_csv('vod_add_id.csv')
vod_list

,Unnamed: 0,name,Summary,actors,director,BigCategory,SmallCategory,Category,rename,counts,vod_name,running_time,vod_id
0,0,와이낫크루 뜻밖의 여행,가는 길이 좀 멀고 험해도 나만의 취저(취향 저격)인 곳은 집요하게 찾아가는 MZ를...,"KCM,권은비,조정식,박현규","은지향, 김한진, 김남훈, 명민아, 김주희",TV 연예/오락,기타,TV프로그램,와이낫크루뜻밖의여행,16,와이낫크루: 뜻밖의 여행,69분,0
1,1,그레이트 뷰티,40여 년 전 소설 한 권을 끝으로 더 이상 책을 쓰지 못하는 젭은 로마 1%의 삶...,"토니 세르빌로,사브리나 페릴리,세레나 그랜디,이사벨라 페라리",파올로 소렌티노,영화,기타,영화,그레이트뷰티,1,그레이트 뷰티,141분,1
2,2,해안선,평온해 보이는 동해안의 바닷가. “경고! 밤 7시 이후 이곳을 접근하는 자는 간첩으...,"장동건,김정학,박지아,유해진",김기덕,영화,액션/어드벤쳐,영화,해안선,1,해안선,94분,2
3,3,아는 형님,이성 상실 본능 충실 형님학교에서 벌어지는 세상의 모든 놀이 [아는 형님],"강호동,서장훈,김영철,이수근,김희철,민경훈,이상민","민철기, 최창수, 이수아, 황선영, 지은정",TV 연예/오락,기타,TV프로그램,아는형님,44,아는 형님,87분,3
4,4,베놈,진실을 위해서라면 몸을 사리지 않는 정의로운 열혈 기자 '에디 브록' 거대 기업 라...,"톰 하디,미셀 윌리엄스,리즈 아메드,제니 슬레이트,레이드 스콧,스콧 헤이즈,우디 해...",루벤 플레셔,영화,액션/어드벤쳐,영화,베놈,2,베놈,107분,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4710,4710,베놈 2: 렛 데어 비 카니지,‘베놈’과 완벽한 파트너가 된 ‘에디 브록’(톰 하디) 앞에 ‘클리터스 캐서디’(우...,"톰 하디,미셀 윌리엄스,나오미 해리스,레이드 스콧,스티븐 그레이엄,우디 해럴슨",앤디 서키스,영화,액션/어드벤쳐,영화,베놈2:렛데어비카니지,11,베놈 2: 렛 데어 비 카니지,97분,4710
4711,4711,성월동화,야쿠자 두목 사토는 대도 당걸에게 자신의 일기를 찾도록 의뢰한다. 그러나 당걸에게 ...,"장국영,토키와 타카코,양자경,이찬삼,고첩,혜천사,요계지,황가락",진가상,영화,멜로,영화,성월동화,3,성월동화 2 - 연전충승,98분,4711
4712,4712,다큐인사이트 코드블루,히포크라테스 선서를 외치고 의사들이 출발선에 서는 3월. 전남대병원에 발 디딘 새내...,유승호,-,TV 시사/교양,기타,TV프로그램,다큐인사이트코드블루,1,다큐인사이트 코드블루,47분,4712
4713,4713,캠프 제로베이스원,9인만의 스페셜 에디션 캠핑카를 타고 떠나는 예능 프로그램,제로베이스원,서예진,TV 연예/오락,기타,TV프로그램,캠프제로베이스원,1,캠프 제로베이스원,109분,4713


In [28]:
# BigCategory, SmallCategory로 장르 생성
vod_list['genres'] = vod_list['BigCategory'].str.replace('/', '').str.replace(' ', '') + ' ' + vod_list['SmallCategory'].str.replace('/', '').str.replace(' ', '')
vod_list

,Unnamed: 0,name,Summary,actors,director,BigCategory,SmallCategory,Category,rename,counts,vod_name,running_time,vod_id,genres
0,0,와이낫크루 뜻밖의 여행,가는 길이 좀 멀고 험해도 나만의 취저(취향 저격)인 곳은 집요하게 찾아가는 MZ를...,"KCM,권은비,조정식,박현규","은지향, 김한진, 김남훈, 명민아, 김주희",TV 연예/오락,기타,TV프로그램,와이낫크루뜻밖의여행,16,와이낫크루: 뜻밖의 여행,69분,0,TV연예오락 기타
1,1,그레이트 뷰티,40여 년 전 소설 한 권을 끝으로 더 이상 책을 쓰지 못하는 젭은 로마 1%의 삶...,"토니 세르빌로,사브리나 페릴리,세레나 그랜디,이사벨라 페라리",파올로 소렌티노,영화,기타,영화,그레이트뷰티,1,그레이트 뷰티,141분,1,영화 기타
2,2,해안선,평온해 보이는 동해안의 바닷가. “경고! 밤 7시 이후 이곳을 접근하는 자는 간첩으...,"장동건,김정학,박지아,유해진",김기덕,영화,액션/어드벤쳐,영화,해안선,1,해안선,94분,2,영화 액션어드벤쳐
3,3,아는 형님,이성 상실 본능 충실 형님학교에서 벌어지는 세상의 모든 놀이 [아는 형님],"강호동,서장훈,김영철,이수근,김희철,민경훈,이상민","민철기, 최창수, 이수아, 황선영, 지은정",TV 연예/오락,기타,TV프로그램,아는형님,44,아는 형님,87분,3,TV연예오락 기타
4,4,베놈,진실을 위해서라면 몸을 사리지 않는 정의로운 열혈 기자 '에디 브록' 거대 기업 라...,"톰 하디,미셀 윌리엄스,리즈 아메드,제니 슬레이트,레이드 스콧,스콧 헤이즈,우디 해...",루벤 플레셔,영화,액션/어드벤쳐,영화,베놈,2,베놈,107분,4,영화 액션어드벤쳐
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4710,4710,베놈 2: 렛 데어 비 카니지,‘베놈’과 완벽한 파트너가 된 ‘에디 브록’(톰 하디) 앞에 ‘클리터스 캐서디’(우...,"톰 하디,미셀 윌리엄스,나오미 해리스,레이드 스콧,스티븐 그레이엄,우디 해럴슨",앤디 서키스,영화,액션/어드벤쳐,영화,베놈2:렛데어비카니지,11,베놈 2: 렛 데어 비 카니지,97분,4710,영화 액션어드벤쳐
4711,4711,성월동화,야쿠자 두목 사토는 대도 당걸에게 자신의 일기를 찾도록 의뢰한다. 그러나 당걸에게 ...,"장국영,토키와 타카코,양자경,이찬삼,고첩,혜천사,요계지,황가락",진가상,영화,멜로,영화,성월동화,3,성월동화 2 - 연전충승,98분,4711,영화 멜로
4712,4712,다큐인사이트 코드블루,히포크라테스 선서를 외치고 의사들이 출발선에 서는 3월. 전남대병원에 발 디딘 새내...,유승호,-,TV 시사/교양,기타,TV프로그램,다큐인사이트코드블루,1,다큐인사이트 코드블루,47분,4712,TV시사교양 기타
4713,4713,캠프 제로베이스원,9인만의 스페셜 에디션 캠핑카를 타고 떠나는 예능 프로그램,제로베이스원,서예진,TV 연예/오락,기타,TV프로그램,캠프제로베이스원,1,캠프 제로베이스원,109분,4713,TV연예오락 기타


In [29]:
vod_list[vod_list['vod_id'] == 160]

,Unnamed: 0,name,Summary,actors,director,BigCategory,SmallCategory,Category,rename,counts,vod_name,running_time,vod_id,genres
160,160,가비,"1896년, 고종(박희순)이 러시아 공사관으로 피신(아관파천)해 대한제국을 준비하던...","주진모,김소연,박희순,유선,조덕현,김응수,조경훈,조승연",장윤현,영화,드라마,영화,가비,3,가비,115분,160,영화 드라마


rating 정보에 vod_name 추가

In [32]:
ratings_id = pd.merge(ratings, vod_list[['vod_name', 'vod_id']], on='vod_id', how='left')
ratings_id

,subsr,vod_id,rating,stars,vod_name
0,59879000,1190,0.155703,1,"신성한, 이혼"
1,59879000,2092,0.204679,2,소방서 옆 경찰서 그리고 국과수
2,59895000,784,1.000000,5,금이야 옥이야
3,59900000,38,0.001905,1,미운 우리 새끼
4,59900000,149,0.490826,3,마파도
...,...,...,...,...,...
2166,67148000,689,0.091667,1,잠자는 숲속의 공주
2167,67148000,880,0.931818,5,타요의 씽씽극장 동요2
2168,67154000,476,0.009160,1,스트릿 우먼 파이터2
2169,67161000,3824,0.001754,1,스파이 코드명 포춘


In [33]:
ratings_id.to_csv('ratings.csv', encoding='utf-8')

In [34]:
vod_list

,Unnamed: 0,name,Summary,actors,director,BigCategory,SmallCategory,Category,rename,counts,vod_name,running_time,vod_id,genres
0,0,와이낫크루 뜻밖의 여행,가는 길이 좀 멀고 험해도 나만의 취저(취향 저격)인 곳은 집요하게 찾아가는 MZ를...,"KCM,권은비,조정식,박현규","은지향, 김한진, 김남훈, 명민아, 김주희",TV 연예/오락,기타,TV프로그램,와이낫크루뜻밖의여행,16,와이낫크루: 뜻밖의 여행,69분,0,TV연예오락 기타
1,1,그레이트 뷰티,40여 년 전 소설 한 권을 끝으로 더 이상 책을 쓰지 못하는 젭은 로마 1%의 삶...,"토니 세르빌로,사브리나 페릴리,세레나 그랜디,이사벨라 페라리",파올로 소렌티노,영화,기타,영화,그레이트뷰티,1,그레이트 뷰티,141분,1,영화 기타
2,2,해안선,평온해 보이는 동해안의 바닷가. “경고! 밤 7시 이후 이곳을 접근하는 자는 간첩으...,"장동건,김정학,박지아,유해진",김기덕,영화,액션/어드벤쳐,영화,해안선,1,해안선,94분,2,영화 액션어드벤쳐
3,3,아는 형님,이성 상실 본능 충실 형님학교에서 벌어지는 세상의 모든 놀이 [아는 형님],"강호동,서장훈,김영철,이수근,김희철,민경훈,이상민","민철기, 최창수, 이수아, 황선영, 지은정",TV 연예/오락,기타,TV프로그램,아는형님,44,아는 형님,87분,3,TV연예오락 기타
4,4,베놈,진실을 위해서라면 몸을 사리지 않는 정의로운 열혈 기자 '에디 브록' 거대 기업 라...,"톰 하디,미셀 윌리엄스,리즈 아메드,제니 슬레이트,레이드 스콧,스콧 헤이즈,우디 해...",루벤 플레셔,영화,액션/어드벤쳐,영화,베놈,2,베놈,107분,4,영화 액션어드벤쳐
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4710,4710,베놈 2: 렛 데어 비 카니지,‘베놈’과 완벽한 파트너가 된 ‘에디 브록’(톰 하디) 앞에 ‘클리터스 캐서디’(우...,"톰 하디,미셀 윌리엄스,나오미 해리스,레이드 스콧,스티븐 그레이엄,우디 해럴슨",앤디 서키스,영화,액션/어드벤쳐,영화,베놈2:렛데어비카니지,11,베놈 2: 렛 데어 비 카니지,97분,4710,영화 액션어드벤쳐
4711,4711,성월동화,야쿠자 두목 사토는 대도 당걸에게 자신의 일기를 찾도록 의뢰한다. 그러나 당걸에게 ...,"장국영,토키와 타카코,양자경,이찬삼,고첩,혜천사,요계지,황가락",진가상,영화,멜로,영화,성월동화,3,성월동화 2 - 연전충승,98분,4711,영화 멜로
4712,4712,다큐인사이트 코드블루,히포크라테스 선서를 외치고 의사들이 출발선에 서는 3월. 전남대병원에 발 디딘 새내...,유승호,-,TV 시사/교양,기타,TV프로그램,다큐인사이트코드블루,1,다큐인사이트 코드블루,47분,4712,TV시사교양 기타
4713,4713,캠프 제로베이스원,9인만의 스페셜 에디션 캠핑카를 타고 떠나는 예능 프로그램,제로베이스원,서예진,TV 연예/오락,기타,TV프로그램,캠프제로베이스원,1,캠프 제로베이스원,109분,4713,TV연예오락 기타


In [35]:
vod_list.to_csv('vod_add_id.csv', encoding='utf-8')

In [36]:
# 장르 정보 추출
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
genres = cv.fit_transform(vod_list.genres)

cv.vocabulary_

{'tv연예오락': 5,
 '기타': 8,
 '영화': 29,
 '액션어드벤쳐': 26,
 '코미디': 38,
 'tv시사교양': 3,
 'tv드라마': 2,
 '외화시리즈': 31,
 '키즈': 39,
 '오락': 30,
 '애니메이션': 24,
 'tv애니메이션': 4,
 '명랑코믹': 18,
 '미분류': 22,
 '인물다큐': 35,
 '멜로': 17,
 '학원순정연애': 41,
 '드라마': 13,
 'sf메카': 0,
 '라이프': 14,
 '운동건강': 33,
 '공포스릴러': 7,
 '스포츠': 23,
 '레슬링': 15,
 '다큐': 10,
 '인물': 34,
 '역사': 27,
 '다큐멘터리': 11,
 'sf환타지': 1,
 '단편': 12,
 '액션모험': 25,
 '로맨틱코미디': 16,
 '자연': 36,
 '학습': 40,
 '무협': 19,
 '추리미스터리': 37,
 '무협환타지': 20,
 '공연음악': 6,
 '뮤지컬': 21,
 '우리동네': 32,
 '연예오락': 28,
 '뉴스': 9}

In [37]:
# one-hot vector 생성
genres = pd.DataFrame(genres.toarray(), columns=list(sorted(cv.vocabulary_.keys(), key=lambda x:cv.vocabulary_[x])))
genres

,sf메카,sf환타지,tv드라마,tv시사교양,tv애니메이션,tv연예오락,공연음악,공포스릴러,기타,뉴스,...,우리동네,운동건강,인물,인물다큐,자연,추리미스터리,코미디,키즈,학습,학원순정연애
0,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4710,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4711,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4712,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4713,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


각 contents 간 거리 계산

In [38]:
from sklearn.neighbors import NearestNeighbors

# n_neighbors: 가장 가까운 n개의 이웃을 찾도록 지정합니다.
nbrs = NearestNeighbors(n_neighbors=100).fit(genres)
nbrs

NearestNeighbors(n_neighbors=100)

In [39]:
# 0번째 영화와 1번째 영화의 유클리디언 거리를 계산하기
a = genres.iloc[0, :]
print(a)
b = genres.iloc[1, :]
print(b)
np.linalg.norm(a-b)

sf메카       0
sf환타지      0
tv드라마      0
tv시사교양     0
tv애니메이션    0
tv연예오락     1
공연음악       0
공포스릴러      0
기타         1
뉴스         0
다큐         0
다큐멘터리      0
단편         0
드라마        0
라이프        0
레슬링        0
로맨틱코미디     0
멜로         0
명랑코믹       0
무협         0
무협환타지      0
뮤지컬        0
미분류        0
스포츠        0
애니메이션      0
액션모험       0
액션어드벤쳐     0
역사         0
연예오락       0
영화         0
오락         0
외화시리즈      0
우리동네       0
운동건강       0
인물         0
인물다큐       0
자연         0
추리미스터리     0
코미디        0
키즈         0
학습         0
학원순정연애     0
Name: 0, dtype: int64
sf메카       0
sf환타지      0
tv드라마      0
tv시사교양     0
tv애니메이션    0
tv연예오락     0
공연음악       0
공포스릴러      0
기타         1
뉴스         0
다큐         0
다큐멘터리      0
단편         0
드라마        0
라이프        0
레슬링        0
로맨틱코미디     0
멜로         0
명랑코믹       0
무협         0
무협환타지      0
뮤지컬        0
미분류        0
스포츠        0
애니메이션      0
액션모험       0
액션어드벤쳐     0
역사         0
연예오락       0
영화         1
오락         0
외화시리즈      0
우리동네       0
운동건

1.4142135623730951

In [68]:
distances, indices = nbrs.kneighbors([a])
distances
indices
recommendations = vod_list.loc[indices[0], ["vod_id"]]
recommendations
recommendations["distance"] = distances[0]
recommendations

C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,vod_id,distance
1700,1700,0.0
240,240,0.0
239,239,0.0
2029,2029,0.0
2030,2030,0.0
...,...,...
1714,1714,0.0
1715,1715,0.0
3667,3667,0.0
3742,3742,0.0


장르 유사한 100개에 평점 정보 결합

In [42]:
# asset_nm 별로 평점 평균, 평점 개수 계산
ratings_mean_count = ratings_id.groupby('vod_id').agg({"stars": ["mean", "count"]})
ratings_mean_count.columns = ["ratings_mean", "ratings_count"]

# 평점은 소수점 1자리에서 반올림
ratings_mean_count.ratings_mean = ratings_mean_count.ratings_mean.round(1)
ratings_mean_count

,ratings_mean,ratings_count
vod_id,,
0,1.0,1
3,2.0,2
6,2.3,3
7,2.0,15
8,3.4,21
...,...,...
4683,5.0,1
4689,1.0,1
4696,2.0,2


In [43]:
# 기존 추천 데이터에 합쳐줍니다
recommendations = pd.merge(recommendations, ratings_mean_count, left_index=True, right_index=True, how="left")
# 거리순에 이은 평점, 평점수 순으로 정렬해줍니다.
recommendations.sort_values(["distance", "ratings_mean", "ratings_count"], ascending=[True, False, False], inplace=True)
recommendations

,vod_id,distance,ratings_mean,ratings_count
3124,3124,0.0,5.0,1.0
1090,1090,0.0,5.0,1.0
3838,3838,0.0,4.0,1.0
320,320,0.0,4.0,1.0
836,836,0.0,4.0,1.0
...,...,...,...,...
1465,1465,0.0,NaN,NaN
4069,4069,0.0,NaN,NaN
3645,3645,0.0,NaN,NaN
1464,1464,0.0,NaN,NaN


사용자별 추천

In [44]:
ratings_id

,subsr,vod_id,rating,stars,vod_name
0,59879000,1190,0.155703,1,"신성한, 이혼"
1,59879000,2092,0.204679,2,소방서 옆 경찰서 그리고 국과수
2,59895000,784,1.000000,5,금이야 옥이야
3,59900000,38,0.001905,1,미운 우리 새끼
4,59900000,149,0.490826,3,마파도
...,...,...,...,...,...
2166,67148000,689,0.091667,1,잠자는 숲속의 공주
2167,67148000,880,0.931818,5,타요의 씽씽극장 동요2
2168,67154000,476,0.009160,1,스트릿 우먼 파이터2
2169,67161000,3824,0.001754,1,스파이 코드명 포춘


In [133]:
# 59879000 계정이 본 vod_id
user = 67164000
ratings_id[ratings_id['subsr'] == user]['vod_id']
# vod_id = ratings_id[ratings_id['subsr'] == 59879000]['vod_id'][0]

2170    2865
Name: vod_id, dtype: int64

In [152]:
# 한 계정이 여러 개 봤을 때 각각의 추천 모두 가져옴
# 계정 list
users = ratings_id['subsr'].unique()
recom_list = dict()
for user in users:
    recommendations = pd.DataFrame()
    for vod_id in ratings_id[ratings_id['subsr'] == user]['vod_id']:
        a = genres.iloc[vod_id, :]
        distances, indexes = nbrs.kneighbors([a])
        recom = vod_list.loc[indexes[0], ["vod_name", 'vod_id', 'Category']]
        recom["distance"] = distances[0]
        recommendations = pd.concat([recommendations, recom])
    # 기존 추천 데이터에 합쳐줍니다
    recommendations = pd.merge(recommendations, ratings_mean_count, left_index=True, right_index=True, how="left")
    # 거리순에 이은 평점, 평점수 순으로 정렬해줍니다.
    recommendations.sort_values(["distance", "ratings_mean", "ratings_count"], ascending=[True, False, False], inplace=True)
    # 중복제거
    recommendations = recommendations.drop_duplicates(subset=['vod_id'])
    recommendations['list'] = recommendations['vod_name'] + '/' + recommendations['Category']
    recom = ', '.join(recommendations['list'][:10].tolist())
    recom_list[user] = recom

C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have val

C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have val

C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have val

C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have val

C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have val

...

C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have val

C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have val

C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have val

C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have val

C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\base.py:420: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


In [130]:
# 기존 추천 데이터에 합쳐줍니다
recommendations = pd.merge(recommendations, ratings_mean_count, left_index=True, right_index=True, how="left")
# 거리순에 이은 평점, 평점수 순으로 정렬해줍니다.
recommendations.sort_values(["distance", "ratings_mean", "ratings_count"], ascending=[True, False, False], inplace=True)
# 중복제거
recommendations = recommendations.drop_duplicates(subset=['vod_id'])
recommendations['subsr'] = 59879000
recommendations['vod_name'][:10].tolist()

['일급 살수',
 '극장판 주술회전 0',
 '정복자 윌리엄',
 '드론: 미래전쟁',
 '천하제패 : 반란의 시대',
 '오공',
 '현장법사: 서유기의 시작',
 '살수',
 '블랙 아담',
 '트랜스포머: 비스트의 서막']

In [137]:
# 기존 추천 데이터에 합쳐줍니다
recommendations2 = pd.merge(recommendations, ratings_mean_count, left_index=True, right_index=True, how="left")
# 거리순에 이은 평점, 평점수 순으로 정렬해줍니다.
recommendations2.sort_values(["distance", "ratings_mean", "ratings_count"], ascending=[True, False, False], inplace=True)
# 중복제거
recommendations2 = recommendations.drop_duplicates(subset=['vod_id'])
# 계정당 상위 10개 추출
recommendations2
recommendations2['list'] = recommendations2['vod_name'] + '/' + recommendations2['Category']


,vod_name,vod_id,Category,distance,list
1700,개와 늑대의 시간,1700,TV프로그램,0.0,개와 늑대의 시간/TV프로그램
240,불굴의 며느리,240,TV프로그램,0.0,불굴의 며느리/TV프로그램
239,이 죽일놈의 사랑,239,TV프로그램,0.0,이 죽일놈의 사랑/TV프로그램
2029,전원일기,2029,TV프로그램,0.0,전원일기/TV프로그램
2030,동네변호사 조들호2 미리보기,2030,TV프로그램,0.0,동네변호사 조들호2 미리보기/TV프로그램
...,...,...,...,...,...
1714,지운수대통,1714,TV프로그램,0.0,지운수대통/TV프로그램
1715,그래도 당신,1715,TV프로그램,0.0,그래도 당신/TV프로그램
3667,밀회,3667,TV프로그램,0.0,밀회/TV프로그램
3742,내 이름은 김삼순,3742,TV프로그램,0.0,내 이름은 김삼순/TV프로그램


In [148]:
a = ', '.join(recommendations2['list'][:10].tolist())
a

'개와 늑대의 시간/TV프로그램, 불굴의 며느리/TV프로그램, 이 죽일놈의 사랑/TV프로그램, 전원일기/TV프로그램, 동네변호사 조들호2 미리보기/TV프로그램, 모범형사2/TV프로그램, 스토브리그 파이널 리포트/TV프로그램, 그냥 사랑하는 사이/TV프로그램, 조선왕조 500년 인현황후/TV프로그램, 조선왕조 500년 한중록/TV프로그램'

In [117]:
result2

{59895000: 305              왔다! 장보리
 958             우당탕탕 패밀리
 2994    이번주, 아내가 바람을 핍니다
 315                   은희
 535          어쩌다 마주친, 그대
 796                  일지매
 991              오자룡이 간다
 955               야경꾼 일지
 1713          사임당, 빛의 일기
 515             법대로 사랑하라
 Name: vod_name, dtype: object}

In [153]:
result = pd.DataFrame(list(recom_list.items()), columns=['subsr', 'vod_recommendations'])
result

,subsr,vod_recommendations
0,59879000,"왔다! 장보리/TV프로그램, 우당탕탕 패밀리/TV프로그램, 이번주, 아내가 바람을 ..."
1,59895000,"이번주, 아내가 바람을 핍니다/TV프로그램, 우당탕탕 패밀리/TV프로그램, 왔다! ..."
2,59900000,"도망쳐-악마와 손절하는 완벽한 타이밍/TV프로그램, 드론: 미래전쟁/영화, TV쇼 ..."
3,59921000,"올빼미/영화, 특등비서/영화, 엘레지의 여왕/영화, 엄마 아빠 오래 사세요/영화, ..."
4,59930000,"왔다! 장보리/TV프로그램, 우당탕탕 패밀리/TV프로그램, 특등비서/영화, 엘레지의..."
...,...,...
401,67140000,"타요의 씽씽극장 동요2/키즈, 장화 신은 고양이/키즈, 늑대와 일곱 마리 아기 염소..."
402,67148000,"타요의 씽씽극장 동요2/키즈, 장화 신은 고양이/키즈, 늑대와 일곱 마리 아기 염소..."
403,67154000,"안하던 짓을 하고 그래/TV프로그램, 벌거벗은 세계사/TV프로그램, 전국노래자랑/T..."
404,67161000,"일급 살수/영화, 극장판 주술회전 0/영화, 정복자 윌리엄/영화, 드론: 미래전쟁/..."


In [154]:
result.to_csv('genre_based_recommendation.csv')